> Objetivo: Corrigir os tipos de dados das colunas, inferschema desabilitado para o propósito da ATV

In [0]:
user = str(dbutils.fs.ls("file:/Workspace/Users/")).split('/')[3]
filespath = "file:/Workspace/Users/" + f"{user}/treinamento_formula1/landing"
savepath = "dbfs:/mnt/formula1/bronze/circuit"

dataframe = spark.read.csv(filespath+"/circuito.csv", header=True)
dfDict = {
    'circuitId': 'integer',
    'circuitRef': 'string',
    'name': 'string',
    'location': 'string',
    'country': 'string',
    'lat': 'double',
    'lng': 'double',
    'alt': 'integer',
    'url': 'string'
}

def cast_columntypes(df):
    for col, dtype in dfDict.items():
        df = alter_type(df, col, dtype)
    return df

def alter_type(df, coluna, tipo):
    return df.withColumn(coluna, df[coluna].cast(tipo))

dataframe = cast_columntypes(dataframe)


In [0]:
print(user)
print('\nAlterando colunas....')
dataframe.printSchema()


print('Criando pasta no dbfs...')
dbutils.fs.mkdirs(savepath)
dbutils.fs.rm(savepath + '/circuitoRaw.delta', True)
print('Escrevendo arquivo...')
dataframe.write.format('delta').save(f'{savepath}/circuitoRaw.delta')